# 004-Running kmeans on snowflake

In [1]:
import snowflake.connector
from src.util import *
from src.kmeans_sf import *
import numpy as np
import os
import pandas as pd

### start snowflake connector

In [2]:
cred = "sf-cred.txt"
f = open(cred, "r")
USER, PW, ACC, DATABASE, WAREHOUSE = f.read().split()

In [3]:
ctx = snowflake.connector.connect(
    user=USER,
    password=PW,
    account=ACC,
    database=DATABASE,
    warehouse=WAREHOUSE
    )
cs = ctx.cursor()

### a function to run on each dataset

In [4]:
def dataset_run(con, name):
    d = dataset(name, 'snowflake', con, "results/snowflake")
    d.load_dataset(load_dataset_sf, "s3://data516project/datasets/")
    t = d.kmeans_train(kmeans_sf, 100)
    d.save_output()
    del d
    return t

### obtain all dataset names

In [5]:
datasets = [name.split('.')[0] for name in os.listdir("datasets/") if '-' not in name]
datasets

['a3',
 'a3e1',
 'a3e2',
 'a3e3',
 'dim032',
 'dim032e1',
 'dim032e2',
 'dim032e3',
 'dim128',
 'dim128e1',
 'dim128e2',
 'dim128e3',
 'dim512',
 'dim512e1',
 'dim512e2',
 'dim512e3']

### run each dataset and save time to a dataframe

In [6]:
cols = ['dataset', 'time']
if not os.path.exists('results/time_snowflake.csv'):
    df = pd.DataFrame(columns=cols)
else:
    df = pd.read_csv('results/time_snowflake.csv')

In [8]:
for name in [dataset for dataset in datasets if 'e3' not in dataset]:
    if name in list(df['dataset']):
        print("#### "+name+" skipped")
        continue
    t = dataset_run(cs, name)
    tmp = pd.DataFrame([[name, t]], columns=cols)
    df = df.append(tmp, ignore_index=True)
    df.to_csv('results/time_snowflake.csv', index=False)

#### a3 skipped
#### a3e1 skipped
#### a3e2 skipped
#### dim032
- dim032 initiated
- dim032 data loaded
- dim032 training start



- dim032 trained
time used: 227974.169ms
- dim032 saved
#### dim032e1
- dim032e1 initiated
- dim032e1 data loaded
- dim032e1 training start



- dim032e1 trained
time used: 437876.67ms
- dim032e1 saved
#### dim032e2
- dim032e2 initiated
- dim032e2 data loaded
- dim032e2 training start



- dim032e2 trained
time used: 3620311.479ms
- dim032e2 saved
#### dim128
- dim128 initiated
- dim128 data loaded
- dim128 training start



- dim128 trained
time used: 274883.249ms
- dim128 saved
#### dim128e1
- dim128e1 initiated
- dim128e1 data loaded
- dim128e1 training start



- dim128e1 trained
time used: 1524318.141ms
- dim128e1 saved
#### dim128e2
- dim128e2 initiated
- dim128e2 data loaded
- dim128e2 training start


KeyboardInterrupt: 

In [9]:
for name in [dataset for dataset in datasets if '512' in dataset]:
    if name in list(df['dataset']):
        print("#### "+name+" skipped")
        continue
    t = dataset_run(cs, name)
    tmp = pd.DataFrame([[name, t]], columns=cols)
    df = df.append(tmp, ignore_index=True)
    df.to_csv('results/time_snowflake.csv', index=False)

#### dim512
- dim512 initiated
- dim512 data loaded
- dim512 training start



- dim512 trained
time used: 611197.106ms
- dim512 saved
#### dim512e1
- dim512e1 initiated
- dim512e1 data loaded
- dim512e1 training start


ProgrammingError: 100132 (P0000): JavaScript execution error: UDF execution time limit exceeded by function FIND_ID

In [ ]:
ctx.close()